### Neural Style Transfer

In [1]:
from keras.preprocessing.image import load_img , img_to_array

Using TensorFlow backend.


In [2]:
target_path = 'Photo 2.jpg'
style_path = 'Style2.jpg'

width , height = load_img(target_path).size
img_height = 800
img_width = int(width * img_height / height)

In [3]:
img_width

800

In [4]:
from keras.applications import vgg16
import numpy as np

In [5]:
def preprocess_image(img_path):
    img = load_img(img_path , target_size=(img_height , img_width))
    img = img_to_array(img)
    img = np.expand_dims(img , axis = 0)
    img = vgg16.preprocess_input(img)
    return img

def deprocess_img(img):
    # remove zero center pixel
    img[:,:,0] += 103.939 
    img[:,:,1] += 116.779
    img[:,:,2] += 123.68

    # BGR to RGB
    img = img[: ,:, ::-1]
    img = np.clip(img , 0 ,255).astype('uint8')
    return img

### Loss Funcitons

Content Loss <br>
Style Loss <br>
Gram matrix <br>
total Variation Loss <br>

In [6]:
import keras.backend as K

In [7]:
def content_loss(base , combination):
    return K.sum(K.square(combination - base))

def gram_matirx(x):
    features = K.batch_flatten(K.permute_dimensions(x , (2,0,1)))
    gram = K.dot(features , K.transpose(features))
    return gram

def style_loss(style , combination):
    S = gram_matirx(style)
    C = gram_matirx(combination)
    channels = 3
    size = img_width * img_height
    return K.sum(K.square(S - C)) / (4.*(channels**2)*(size**2))

def total_variation_loss(x):
    a = K.square(
        x[: , :img_height-1, :img_width-1 ,:] -
        x[: , 1: , :img_width-1 , :]  )
    
    b = K.square(
        x[: , :img_height-1, :img_width-1 ,:] - 
        x[: , img_height - 1: , 1: , :]       )
    
    return K.sum(K.pow(a + b , 1.25))

### Load Model

In [8]:
target_img = K.constant(preprocess_image(target_path))
style_image = K.constant(preprocess_image(style_path))

In [9]:
combinaiton_image = K.placeholder((1,img_height , img_width , 3))

In [10]:
input_tensor = K.concatenate([target_img , style_image , combinaiton_image] , axis = 0)

In [11]:
model = vgg16.VGG16(input_tensor = input_tensor , weights = 'imagenet', include_top = False)
print('Model loaded')

Model loaded


### Final loss minimizing

In [12]:
output_dict = dict([(layer.name , layer.output) for layer in model.layers])

In [13]:
content_layer = 'block5_conv2'

style_layer = ['block1_conv1',
               'block2_conv1',
               'block3_conv1',
               'block4_conv1',
               'block5_conv1']

In [14]:
total_variation_weight = 1e-4
content_weight = 0.025
style_weight = 1.

In [15]:
loss = K.variable(0)
layer_features = output_dict[content_layer]

target_image_features = layer_features[0 ,: ,: ,:]
combination_image_features = layer_features[2 , : ,: ,:]
loss = loss + content_weight*content_loss(target_image_features , combination_image_features)

In [16]:
for layer_name in style_layer:
    layer_features = output_dict[layer_name]
    style_reference_features = layer_features[1 , : , : , :]
    combinational_features = layer_features[2, : , : ,:]
    sl = style_loss(style_reference_features , combinational_features)
    loss = loss + ((style_weight/len(style_layer))*sl)

loss = loss + total_variation_weight*total_variation_loss(combinaiton_image)

### Compute gradients

In [17]:
# compute gradients of generated image wrt loss
grads = K.gradients(loss , combinaiton_image)[0]

In [18]:
# function that fetch current loss and gradients
fetch_loss_and_grads = K.function([combinaiton_image],[loss, grads])

In [19]:
class Evaluator(object):
    
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    
    def loss(self, x):
        assert self.loss_value is None
        x = x.reshape((1, img_height, img_width, 3))
        outs = fetch_loss_and_grads([x])
    
        loss_value = outs[0]
        grad_values = outs[1].flatten().astype('float64')
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    
    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values


In [20]:
evaluator = Evaluator()

In [21]:
from scipy.optimize import fmin_l_bfgs_b
import imageio
import time

In [22]:
iterations = 20
x = preprocess_image(target_path)
x= x.flatten()

In [23]:
import matplotlib.image as image

In [ ]:
for i in range(iterations):
    print('start of iterations ' , i)
    start_time = time.time()
    x , min_val , info = fmin_l_bfgs_b(evaluator.loss , x , fprime = evaluator.grads , maxfun = 20)
    
    print('Current loss value :' , min_val)
    img = x.copy().reshape((img_height , img_width , 3))
    img = deprocess_img(img)
    fname = 'New_image_at_iteration_%d.png'%i
    image.imsave(fname , img)
    print('image saved as ',fname)
    end_time = time.time()
    print('Iteration %d completed in %d'%( i ,end_time - start_time))

start of iterations  0
Current loss value : 682962300.0
image saved as  New_image_at_iteration_0.png
Iteration 0 completed in 534
start of iterations  1
Current loss value : 202333040.0
image saved as  New_image_at_iteration_1.png
Iteration 1 completed in 474
start of iterations  2
Current loss value : 106512400.0
image saved as  New_image_at_iteration_2.png
Iteration 2 completed in 427
start of iterations  3
Current loss value : 66409130.0
image saved as  New_image_at_iteration_3.png
Iteration 3 completed in 424
start of iterations  4
